```
--- Day 14: Regolith Reservoir ---
The distress signal leads you to a giant waterfall! Actually, hang on - the signal seems like it's coming from the waterfall itself, and that doesn't make any sense. However, you do notice a little path that leads behind the waterfall.

Correction: the distress signal leads you behind a giant waterfall! There seems to be a large cave system here, and the signal definitely leads further inside.

As you begin to make your way deeper underground, you feel the ground rumble for a moment. Sand begins pouring into the cave! If you don't quickly figure out where the sand is going, you could quickly become trapped!

Fortunately, your familiarity with analyzing the path of falling material will come in handy here. You scan a two-dimensional vertical slice of the cave above you (your puzzle input) and discover that it is mostly air with structures made of rock.

Your scan traces the path of each solid rock structure and reports the x,y coordinates that form the shape of the path, where x represents distance to the right and y represents distance down. Each path appears as a single line of text in your scan. After the first point of each path, each point indicates the end of a straight horizontal or vertical line to be drawn from the previous point. For example:

498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9
This scan means that there are two paths of rock; the first path consists of two straight lines, and the second path consists of three straight lines. (Specifically, the first path consists of a line of rock from 498,4 through 498,6 and another line of rock from 498,6 through 496,6.)

The sand is pouring into the cave from point 500,0.

Drawing rock as #, air as ., and the source of the sand as +, this becomes:


  4     5  5
  9     0  0
  4     0  3
0 ......+...
1 ..........
2 ..........
3 ..........
4 ....#...##
5 ....#...#.
6 ..###...#.
7 ........#.
8 ........#.
9 #########.
Sand is produced one unit at a time, and the next unit of sand is not produced until the previous unit of sand comes to rest. A unit of sand is large enough to fill one tile of air in your scan.

A unit of sand always falls down one step if possible. If the tile immediately below is blocked (by rock or sand), the unit of sand attempts to instead move diagonally one step down and to the left. If that tile is blocked, the unit of sand attempts to instead move diagonally one step down and to the right. Sand keeps moving as long as it is able to do so, at each step trying to move down, then down-left, then down-right. If all three possible destinations are blocked, the unit of sand comes to rest and no longer moves, at which point the next unit of sand is created back at the source.

So, drawing sand that has come to rest as o, the first unit of sand simply falls straight down and then stops:

......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
......o.#.
#########.
The second unit of sand then falls straight down, lands on the first one, and then comes to rest to its left:

......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
.....oo.#.
#########.
After a total of five units of sand have come to rest, they form this pattern:

......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
......o.#.
....oooo#.
#########.
After a total of 22 units of sand:

......+...
..........
......o...
.....ooo..
....#ooo##
....#ooo#.
..###ooo#.
....oooo#.
...ooooo#.
#########.
Finally, only two more units of sand can possibly come to rest:

......+...
..........
......o...
.....ooo..
....#ooo##
...o#ooo#.
..###ooo#.
....oooo#.
.o.ooooo#.
#########.
Once all 24 units of sand shown above have come to rest, all further sand flows out the bottom, falling into the endless void. Just for fun, the path any new sand takes before falling forever is shown here with ~:

.......+...
.......~...
......~o...
.....~ooo..
....~#ooo##
...~o#ooo#.
..~###ooo#.
..~..oooo#.
.~o.ooooo#.
~#########.
~..........
~..........
~..........
Using your scan, simulate the falling sand. How many units of sand come to rest before sand starts flowing into the abyss below?
```

In [7]:
import numpy as np
data = open('inputs/14.input').read().splitlines()
paths = [[tuple(map(int,e.split(','))) for e in r.split(' -> ')] for r in data]


In [8]:

class SandSim:
    AIR=0
    SAND=1
    ROCK=2
    ORIGIN=-1
    SAND_ORIGIN = (500,0)
    
    def __init__(self,paths,infinite=False):    
        boundaries = np.quantile(np.array([
            np.quantile(p,(0,1),0) for p in paths]+[[self.SAND_ORIGIN]*2]),(0,1),(0,1))
        
        (self.x_min,self.x_max),(self.y_min, self.y_max) = boundaries.T
        self.infinite = infinite
        if self.infinite:
            ground_dist = 2
            height = (self.y_max-self.y_min+1)+ground_dist
            self.x_min = min(self.SAND_ORIGIN[0]-height,self.x_min)
            self.x_max = max(self.SAND_ORIGIN[0]+height,self.x_max)
            self.y_max += ground_dist
            paths = paths +[[[self.x_min,self.y_max],[self.x_max,self.y_max]]]
        
        self.sand_map = np.zeros((self.x_max-self.x_min+1,self.y_max-self.y_min+1),dtype=int)
        for path in paths:
            for p in path:
                self.sand_map[p[0]-self.x_min,p[1]-self.y_min] = 1

        lines = (
            np.concatenate([np.linspace(a,b,max(np.abs(np.subtract(b,a)))+1,dtype=int)  
                            for a,b in 
                            zip(p,p[1:])]) 
            for p in paths)
        for line in lines:
            self.sand_map[line[:,0]-self.x_min,line[:,1]-self.y_min] = self.ROCK
            
        self.sand_map[tuple(np.subtract(self.SAND_ORIGIN,(self.x_min,self.y_min)))] = self.SAND
    
    def _next_step(self) -> bool:
        pos = np.subtract(self.SAND_ORIGIN,(self.x_min,self.y_min))
        illegal_substitution = pos
        assert self.sand_map[tuple(illegal_substitution)] == self.SAND

        while(True):
            pos_offs = pos + np.array([[0,1],[-1,1],[1,1]])
            is_illegal = (pos_offs<(0,0)).any(1) | (pos_offs>=(self.sand_map.shape)).any(1)
            pos_offs[is_illegal] = illegal_substitution
            candidates = self.sand_map[tuple(pos_offs.T)] == self.AIR
            case = np.argmax(candidates)
            
            # Illegal move
            if is_illegal.any() and np.argmax(is_illegal | ~candidates) <= case:
                return False
            # No candidates
            if not candidates.any():
                if (pos != illegal_substitution).any():
                    self.sand_map[tuple(pos)] = self.SAND
                    return True
                return False
            pos = pos_offs[case]
            
    def run(self):
        i=0        
        while(self._next_step()): i += 1
        return i + self.infinite
        


In [10]:
def printer(sand_sim):
    for r in sand_sim.sand_map.T:
        print(''.join(['.' if e==SandSim.AIR else '#' if e==SandSim.ROCK else '*' for e in r]))

# infinite = False # Part 1
infinite = True  # Part 2    

sand_sim = SandSim(paths,infinite=True)
print(sand_sim.run())

30214
